# Question and answers with Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

Question Answering (QA) is an important task that involves extracting answers to factual queries posed in natural language. Typically, a QA system processes a query against a knowledge base containing structured or unstructured data and generates a response with accurate information. Ensuring high accuracy is key to developing a useful, reliable and trustworthy question answering system, especially for enterprise use cases. 

Generative AI models like Titan and Claude use probability distributions to generate responses to questions. These models are trained on vast amounts of text data, which allows them to predict what comes next in a sequence or what word might follow a particular word. However, these models are not able to provide accurate or deterministic answers to every question because there is always some degree of uncertainty in the data. Enterprises need to query domain specific and proprietary data and use the information to answer questions, and more generally data on which the model has not been trained on. 

In this module, we will demonstrate how to use the Bedrock Titan model to provide information response to queries.

In this example we will be running the Model with no context and then manually try and provide the context. There is no `RAG` augmentation happening here. This approach works with short documents or single-ton applications, it might not scale to enterprise level question answering where there could be large enterprise documents which cannot all be fit into the prompt sent to the model. 

### Challenges
- How to have the model return factual answers for question

### Proposal
To the above challenges, this notebook proposes the following strategy
#### Prepare documents
Before being able to answer the questions, the documents must be processed and a stored in a document store index
- Here we will send in the request with the full relevant context to the model and expect the response back


## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [71]:
import warnings
warnings.filterwarnings('ignore')

In [72]:
import json
import os
import sys

import boto3
import botocore


module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Section 1: Q&A with the knowledge of the model
In this section we try to use models provided by Bedrock service to answer questions based on the knowledge it gained during the training phase.

In this Notebook we will be using the `invoke_model()` method of Amazon Bedrock client. The mandatory parameters required to use this method are `modelId` which represents the Amazon Bedrock model ARN, and `body` which is the prompt for our task. The `body` prompt changes depending on the foundation model provider selected. We walk through this in detail below

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

In [73]:
context = """18:14:20,387 --- setUpClass() for Flowprobe called ---
18:14:20,387 Temporary dir is /tmp/vpp-unittest-Flowprobe, api socket is /tmp/vpp-unittest-Flowprobe/api.sock
18:14:20,387 Random seed is 1701279145.6510966
18:14:20,388 vpp_cmdline args: ['/w/workspace/vpp-debug-verify-master-ubuntu2204-x86_64/build-root/install-vpp_debug-native/vpp/bin/vpp', 'unix', '{', 'nodaemon', '', 'full-coredump', 'coredump-size unlimited', 'runtime-dir', '/tmp/vpp-unittest-Flowprobe', '}', 'api-trace', '{', 'on', '}', 'api-segment', '{', 'prefix', 'vpp-unittest-Flowprobe', '}', 'cpu', '{', 'main-core', '75', '}', 'physmem', '{', 'max-size', '32m', '}', 'statseg', '{', 'socket-name', '/tmp/vpp-unittest-Flowprobe/stats.sock', '', '}', 'socksvr', '{', 'socket-name', '/tmp/vpp-unittest-Flowprobe/api.sock', '}', 'node { ', '', '}', 'api-fuzz {', 'off', '}', 'plugins', '{', 'plugin', 'dpdk_plugin.so', '{', 'disable', '}', 'plugin', 'rdma_plugin.so', '{', 'disable', '}', 'plugin', 'lisp_unittest_plugin.so', '{', 'enable', '}', 'plugin', 'unittest_plugin.so', '{', 'enable', '}', '}']
18:14:20,388 vpp_cmdline: /w/workspace/vpp-debug-verify-master-ubuntu2204-x86_64/build-root/install-vpp_debug-native/vpp/bin/vpp unix { nodaemon  full-coredump coredump-size unlimited runtime-dir /tmp/vpp-unittest-Flowprobe } api-trace { on } api-segment { prefix vpp-unittest-Flowprobe } cpu { main-core 75 } physmem { max-size 32m } statseg { socket-name /tmp/vpp-unittest-Flowprobe/stats.sock  } socksvr { socket-name /tmp/vpp-unittest-Flowprobe/api.sock } node {   } api-fuzz { off } plugins { plugin dpdk_plugin.so { disable } plugin rdma_plugin.so { disable } plugin lisp_unittest_plugin.so { enable } plugin unittest_plugin.so { enable } }
18:14:20,388 Assigned cpus: [75]
18:14:20,389 Spawned VPP with PID: 373305
18:14:21,722 --- END setUpClass() Flowprobe ---
18:14:21,781 Calling sw_interface_set_flags('flags':<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>,'sw_if_index':1,'context':532382,'_vl_msg_id':704,'client_index':2147483649)
18:14:21,782 Return value: sw_interface_set_flags_reply(_0=705, context=532382, retval=0)
18:14:21,782 API: sw_interface_set_flags (flags=<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>, sw_if_index=2)
18:14:21,782 Calling sw_interface_set_flags('flags':<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>,'sw_if_index':2,'context':532383,'_vl_msg_id':704,'client_index':2147483649)
18:14:21,783 Return value: sw_interface_set_flags_reply(_0=705, context=532383, retval=0)
18:14:21,783 API: sw_interface_set_flags (flags=<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>, sw_if_index=3)
18:14:21,783 Calling sw_interface_set_flags('flags':<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>,'sw_if_index':3,'context':532384,'_vl_msg_id':704,'client_index':2147483649)
18:14:21,784 Return value: sw_interface_set_flags_reply(_0=705, context=532384, retval=0)
18:14:21,784 API: sw_interface_set_flags (flags=<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>, sw_if_index=4)
18:14:21,784 Calling sw_interface_set_flags('flags':<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>,'sw_if_index':4,'context':532385,'_vl_msg_id':704,'client_index':2147483649)
18:14:21,785 Return value: sw_interface_set_flags_reply(_0=705, context=532385, retval=0)
18:14:21,785 API: sw_interface_set_flags (flags=<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>, sw_if_index=5)
18:14:21,785 Calling sw_interface_set_flags('flags':<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>,'sw_if_index':5,'context':532386,'_vl_msg_id':704,'client_index':2147483649)
18:14:21,786 Return value: sw_interface_set_flags_reply(_0=705, context=532386, retval=0)
18:14:21,786 API: sw_interface_set_flags (flags=<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>, sw_if_index=6)
18:14:21,786 Calling sw_interface_set_flags('flags':<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>,'sw_if_index':6,'context':532387,'_vl_msg_id':704,'client_index':2147483649)
18:14:21,787 Return value: sw_interface_set_flags_reply(_0=705, context=532387, retval=0)
18:14:21,787 API: sw_interface_set_flags (flags=<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>, sw_if_index=7)
18:14:21,787 Calling sw_interface_set_flags('flags':<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>,'sw_if_index':7,'context':532388,'_vl_msg_id':704,'client_index':2147483649)
18:14:21,787 Return value: sw_interface_set_flags_reply(_0=705, context=532388, retval=0)
18:14:21,788 API: sw_interface_set_flags (flags=<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>, sw_if_index=8)
18:14:21,788 Calling sw_interface_set_flags('flags':<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>,'sw_if_index':8,'context':532389,'_vl_msg_id':704,'client_index':2147483649)
18:14:21,788 Return value: sw_interface_set_flags_reply(_0=705, context=532389, retval=0)
18:14:21,788 API: sw_interface_set_flags (flags=<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>, sw_if_index=9)
18:14:21,789 Calling sw_interface_set_flags('flags':<vl_api_if_status_flags_t.IF_STATUS_API_FLAG_ADMIN_UP: 1>,'sw_if_index':9,'context':532390,'_vl_msg_id':704,'client_index':2147483649)
18:14:21,789 Return value: sw_interface_set_flags_reply(_0=705, context=532390, retval=0)
18:14:21,789 API: sw_interface_add_del_address (sw_if_index=1, prefix='172.16.1.1/24', is_add=1)
18:14:21,790 Calling sw_interface_add_del_address('sw_if_index':1,'prefix':'172.16.1.1/24','is_add':1,'context':532391,'_vl_msg_id':719,'client_index':2147483649)
18:14:21,792 Return value: sw_interface_add_del_address_reply(_0=720, context=532391, retval=...
18:14:21,792 API: ip_neighbor_add_del (is_add=1, neighbor={'sw_if_index': 1, 'flags': 0, 'mac_address': '02:01:00:00:ff:02', 'ip_address': '172.16.1.2'})
18:14:21,792 Calling ip_neighbor_add_del('is_add':1,'neighbor':{'sw_if_index': 1, 'flags': 0, 'mac_address': '02:01:00:00:ff:02', 'ip_address': '172.16.1.2'},'context':532392,'_vl_msg_id':1040,'client_index':2147483649)
18:14:21,794 Return value: ip_neighbor_add_del_reply(_0=1041, context=532392, retval=0, stats...
18:14:21,794 API: sw_interface_add_del_address (sw_if_index=2, prefix='172.16.2.1/24', is_add=1)
18:14:21,794 Calling sw_interface_add_del_address('sw_if_index':2,'prefix':'172.16.2.1/24','is_add':1,'context':532393,'_vl_msg_id':719,'client_index':2147483649)
18:14:21,796 Return value: sw_interface_add_del_address_reply(_0=720, context=532393, retval=...
18:14:21,796 Sending ARP request for 172.16.2.1 on port pg1
18:14:21,799 CLI: packet-generator new pcap /tmp/vpp-unittest-Flowprobe/pg1_in.pcap source pg1 name pcap1-sw_if_index-2
18:14:21,799 Calling cli_inband('cmd':'packet-generator new pcap /tmp/vpp-unittest-Flowprobe/pg1_in.pcap source pg1 name pcap1-sw_if_index-2\n','context':532394,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,801 Return value: cli_inband_reply(_0=1396, context=532394, retval=0, reply='')
18:14:21,801 Creating hard link /tmp/vpp-unittest-Flowprobe/pg1_in.pcap->/tmp/vpp-unittest-Flowprobe/suiteFlowprobe.[timestamp:1701281661.80118060].pg1-inp-0000.pg1_in.pcap
18:14:21,801 CLI: packet-generator capture pg1 pcap /tmp/vpp-unittest-Flowprobe/pg1_out.pcap disable
18:14:21,801 Calling cli_inband('cmd':'packet-generator capture pg1 pcap /tmp/vpp-unittest-Flowprobe/pg1_out.pcap disable\n','context':532395,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,802 Return value: cli_inband_reply(_0=1396, context=532395, retval=0, reply='')
18:14:21,802 CLI: show packet-generator
18:14:21,802 Calling cli_inband('cmd':'show packet-generator\n','context':532396,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,803 Return value: cli_inband_reply(_0=1396, context=532396, retval=0, reply='Name   ...
18:14:21,803 Unlinking /tmp/vpp-unittest-Flowprobe/pg1_out.pcap
18:14:21,804 CLI: packet-generator capture pg1 pcap /tmp/vpp-unittest-Flowprobe/pg1_out.pcap
18:14:21,804 Calling cli_inband('cmd':'packet-generator capture pg1 pcap /tmp/vpp-unittest-Flowprobe/pg1_out.pcap\n','context':532397,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,805 Return value: cli_inband_reply(_0=1396, context=532397, retval=0, reply='')
18:14:21,805 CLI: clear trace
18:14:21,805 Calling cli_inband('cmd':'clear trace\n','context':532398,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,805 Return value: cli_inband_reply(_0=1396, context=532398, retval=0, reply='')
18:14:21,806 CLI: trace add pg-input 1000
18:14:21,806 Calling cli_inband('cmd':'trace add pg-input 1000\n','context':532399,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,806 Return value: cli_inband_reply(_0=1396, context=532399, retval=0, reply='')
18:14:21,806 CLI: packet-generator enable
18:14:21,807 Calling cli_inband('cmd':'packet-generator enable\n','context':532400,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,807 Return value: cli_inband_reply(_0=1396, context=532400, retval=0, reply='')
18:14:21,807 CLI: show packet-generator
18:14:21,807 Calling cli_inband('cmd':'show packet-generator\n','context':532401,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,809 Return value: cli_inband_reply(_0=1396, context=532401, retval=0, reply='Name   ...
18:14:21,809 CLI: packet-generator delete pcap1-sw_if_index-2
18:14:21,809 Calling cli_inband('cmd':'packet-generator delete pcap1-sw_if_index-2\n','context':532402,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,810 Return value: cli_inband_reply(_0=1396, context=532402, retval=0, reply='')
18:14:21,810 CLI: show trace
18:14:21,810 Calling cli_inband('cmd':'show trace\n','context':532403,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,811 Return value: cli_inband_reply(_0=1396, context=532403, retval=0, reply='-------...
Packet 1

00:00:00:372119: pg-input
  stream 0, 42 bytes, sw_if_index 2
  current data 0, length 42, buffer-pool 0, ref-count 1, trace handle 0x0
  00000000: ffffffffffff02020000ff020806000108000604000102020000ff02ac100202
  00000020: 000000000000ac10020100000000000000000000000000000000000000000000
  00000040: 0000000000000000000000000000000000000000000000000000000000000000
  00000060: 0000000000000000000000000000000000000000000000000000000000000000
00:00:00:372153: ethernet-input
  frame: flags 0x1, hw-if-index 2, sw-if-index 2
  ARP: 02:02:00:00:ff:02 -> ff:ff:ff:ff:ff:ff
00:00:00:372176: l2-input
  l2-input: sw_if_index 2 dst ff:ff:ff:ff:ff:ff src 02:02:00:00:ff:02 [l2-learn l2-flood ]
00:00:00:372184: l2-learn
  l2-learn: sw_if_index 2 dst ff:ff:ff:ff:ff:ff src 02:02:00:00:ff:02 bd_index 1
00:00:00:372195: l2-flood
  l2-flood: sw_if_index 2 dst ff:ff:ff:ff:ff:ff src 02:02:00:00:ff:02 bd_index 1
00:00:00:372202: l2-output
  l2-output: sw_if_index 3 dst ff:ff:ff:ff:ff:ff src 02:02:00:00:ff:02 data 08 06 00 01 08 00 06 04 00 01 02 02
00:00:00:372208: pg2-output
  pg2 flags 0x00180001
  ARP: 02:02:00:00:ff:02 -> ff:ff:ff:ff:ff:ff
  request, type ethernet/IP4, address size 6/4
  02:02:00:00:ff:02/172.16.2.2 -> 00:00:00:00:00:00/172.16.2.1
00:00:00:372219: pg2-tx
    buffer 0x7ffd2: current data 0, length 42, buffer-pool 0, ref-count 1, trace handle 0x0
                    l2-hdr-offset 0 l3-hdr-offset 14 
  ARP: 02:02:00:00:ff:02 -> ff:ff:ff:ff:ff:ff
  request, type ethernet/IP4, address size 6/4
  02:02:00:00:ff:02/172.16.2.2 -> 00:00:00:00:00:00/172.16.2.1

18:14:21,811 CLI: show packet-generator
18:14:21,811 Calling cli_inband('cmd':'show packet-generator\n','context':532404,'_vl_msg_id':1394,'client_index':2147483649)
18:14:21,812 Return value: cli_inband_reply(_0=1396, context=532404, retval=0, reply='no stre...
18:14:21,814 Waiting for capture file /tmp/vpp-unittest-Flowprobe/pg1_out.pcap to appear, timeout is 1s
  File "/usr/lib/python3.10/unittest/runner.py", line 184, in run
    test(result)
   File "/usr/lib/python3.10/unittest/suite.py", line 84, in __call__
    return self.run(*args, **kwds)
   File "/usr/lib/python3.10/unittest/suite.py", line 114, in run
    self._handleClassSetUp(test, result)
   File "/w/workspace/vpp-debug-verify-master-ubuntu2204-x86_64/test/run_tests.py", line 604, in _handleClassSetUp
    super()._handleClassSetUp(test, result)
   File "/usr/lib/python3.10/unittest/suite.py", line 166, in _handleClassSetUp
    setUpClass()
   File "/w/workspace/vpp-debug-verify-master-ubuntu2204-x86_64/test/test_flowprobe.py", line 394, in setUpClass
    super(Flowprobe, cls).setUpClass()
   File "/w/workspace/vpp-debug-verify-master-ubuntu2204-x86_64/test/test_flowprobe.py", line 218, in setUpClass
    cls.pg1.resolve_arp()
   File "/w/workspace/vpp-debug-verify-master-ubuntu2204-x86_64/test/vpp_pg_interface.py", line 536, in resolve_arp
    captured_packet = pg_interface.wait_for_packet(1)
   File "/w/workspace/vpp-debug-verify-master-ubuntu2204-x86_64/test/vpp_pg_interface.py", line 447, in wait_for_packet
    if not self.wait_for_capture_file(timeout):
   File "/w/workspace/vpp-debug-verify-master-ubuntu2204-x86_64/test/vpp_pg_interface.py", line 390, in wait_for_capture_file
    f"{' '.join(format_stack(limit=10))}"

18:14:22,812 Timeout - capture file still nowhere
18:14:22,813 No ARP received on port pg1 """

#### Let's take the whole excerpt and pass it to the model together with the question.

In [74]:
question1 = "What test failed?"
question2 = "What caused the test failure?"
question3 = "How can I get the failed test to pass?"
question4 = "How can I get the failed test to pass without increasing the timeout?"
question5 = "What pcap file was used for the failed test?"
question6 = "Are there any issues with the pcap file?"
question7 = "What VPP command line args were used?"
question8 = "Are the command line arguments to VPP correct?"
question9 = "What is the price of tea in China?"

parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

prompt_data = f"""Answer the question based only on the information provided between ##.
#
{context}
#

Question: {question1}
Answer:"""

##### Invoke the model via boto3 to generate the response

In [75]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

test_flowprobe.py::test_resolve_arp


## Summary

We see the response is a summarized and step by step instruction of how to change the tires . This simple example shows how you can leverage the `RAG` or the Augmentation process to generate a curated response back